In [1]:
!pip install pandas matplotlib seaborn plotly dash ipywidgets

     ---------------------------------------- 7.8/7.8 MB 706.5 kB/s eta 0:00:00


In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import dash
from dash import dcc, html


In [13]:
# Load dataset
data = pd.read_csv('supermarket_sales.csv')  # importing dataset
data.head()  # Display first 5 rows to check data


,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [14]:
# Handle missing values (if any)
data = data.dropna()

# Convert columns to appropriate data types
data['Date'] = pd.to_datetime(data['Date'])


In [15]:
# Extract month and year for analysis
data['Month'] = data['Date'].dt.month
data['Year'] = data['Date'].dt.year

# Ensure 'Time' column is in datetime.time format
data['Time'] = pd.to_datetime(data['Time'], format='%H:%M').dt.time

# Create a combined Year-Month column for trend visualization
data['Year-Month'] = data['Year'].astype(str) + '-' + data['Month'].astype(str).str.zfill(2)

# Check for missing values
print("Missing Values:\n", data.isnull().sum())

Missing Values:
 Invoice ID                 0
Branch                     0
City                       0
Customer type              0
Gender                     0
Product line               0
Unit price                 0
Quantity                   0
Tax 5%                     0
Total                      0
Date                       0
Time                       0
Payment                    0
cogs                       0
gross margin percentage    0
gross income               0
Rating                     0
Month                      0
Year                       0
Year-Month                 0
dtype: int64


In [16]:
import plotly.express as px

# Aggregate total sales by product line
product_sales = data.groupby('Product line')['Total'].sum().reset_index()

# Create a bar chart using Plotly
fig1 = px.bar(product_sales, x='Product line', y='Total', title='Total Sales by Product Line',
              labels={'Total': 'Total Sales ($)', 'Product line': 'Product Line'},
              color='Total', color_continuous_scale='Viridis')

fig1.show()


In [17]:
# Aggregate monthly sales
monthly_sales = data.groupby(['Year-Month'])['Total'].sum().reset_index()

# Create a line chart for sales trends
fig2 = px.line(monthly_sales, x='Year-Month', y='Total', title='Monthly Sales Trends',
               labels={'Total': 'Total Sales ($)', 'Year-Month': 'Month-Year'})

fig2.show()


In [18]:
# Aggregate payment method counts
payment_distribution = data['Payment'].value_counts().reset_index()
payment_distribution.columns = ['Payment Method', 'Count']

# Create a pie chart for payment method distribution
fig3 = px.pie(payment_distribution, names='Payment Method', values='Count',
              title='Payment Method Distribution', hole=0.4)

fig3.show()


In [19]:
# Prepare data for Plotly heatmap
rating_heatmap_data = data.pivot_table(index='City', columns='Product line', values='Rating', aggfunc='mean')

# Create the heatmap using Plotly
import plotly.graph_objects as go

fig4 = go.Figure(data=go.Heatmap(
    z=rating_heatmap_data.values,
    x=rating_heatmap_data.columns,
    y=rating_heatmap_data.index,
    colorscale='Viridis',
    colorbar=dict(title='Avg Rating')
))

fig4.update_layout(
    title='Average Rating by City and Product Line',
    xaxis=dict(title='Product Line'),
    yaxis=dict(title='City')
)


In [22]:
import dash
from dash import dcc, html

# Initialize Dash app
app = dash.Dash(__name__)


# app layout
app.layout = html.Div([
    html.H1('Supermarket Sales Dashboard', style={'text-align': 'center'}),

    html.Div([
        html.H2('Total Sales by Product Line'),
        dcc.Graph(figure=fig1),  # Bar Chart

        html.H2('Monthly Sales Trends'),
        dcc.Graph(figure=fig2),  # Line Chart

        html.H2('Payment Method Distribution'),
        dcc.Graph(figure=fig3),  # Pie Chart

        html.H2('Average Rating by City and Product Line'),
        dcc.Graph(figure=fig4),  # Heatmap
    ])
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False, port=8052)
